## HumanEval and MBPP Code Generation Datasets

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LAION-AI/Open-Assistant/blob/main/data/datasets/codet_humaneval_mbpp/HumanEval_and_MBPP_code_gen.ipynb)

This notebook contains code to parse HumanEval and MBPP Python code generation prompt and solution data and modify to `(prompt, solution)` pairs outputted in a `.jsonl` file.

Requirements: `requests`

In [1]:
import json
from pathlib import Path
import requests

In [2]:
DATA_SOURCES: list[str] = ["HumanEval", "mbpp_sanitized"]
DATA_FILES: list[str] = [f"{source}_for_code_generation.jsonl" for source in DATA_SOURCES]
OUT_FILES: list[str] = [f"{source}_codegen_qa.jsonl" for source in DATA_SOURCES]

IN_DIR = Path("data")
OUT_DIR = Path("data/qa")
OUT_DIR.mkdir(parents=True, exist_ok=True)

FILE_PATHS: list[Path] = [IN_DIR / data_file for data_file in DATA_FILES]
OUT_PATHS: list[Path] = [OUT_DIR / out_file for out_file in OUT_FILES]

In [3]:
def download_file(filename: str):
    url = f"https://raw.githubusercontent.com/microsoft/CodeT/main/CodeT/data/dataset/{filename}"
    response = requests.get(url)
    with open(f"data/{filename}", "wb") as f:
        f.write(response.content)


for filename in DATA_FILES:
    download_file(filename)

We can find the docstring, use its contents as the instruction (prefixed with a prompt) and then use the content prior to the docstring and the canonical solution as the response.

In [4]:
def get_docstring_indices(prompt_lines: list[str]) -> tuple[int, int]:
    docstring_start, docstring_end = None, None

    for i, line in enumerate(prompt_lines):
        if not (line.strip().startswith('"""') or line.strip().startswith("'''")):
            continue
        if docstring_start:
            docstring_end = i
            break
        docstring_start = i

    if docstring_end:
        return docstring_start, docstring_end
    raise ValueError(f"No complete docstring found!\n{prompt_lines}")


def get_before(prompt_lines: list[str], before: int) -> list[str]:
    before_lines = prompt_lines[:before]
    return before_lines


def get_between(prompt_lines: list[str], start: int, end: int) -> list[str]:
    between_lines = prompt_lines[start:end]
    return between_lines

In [5]:
def get_request_and_solution(sample: dict) -> tuple[list[str], list[str]]:
    prompt = sample["prompt"]
    prompt_lines = prompt.splitlines()

    docstring_start, docstring_end = get_docstring_indices(prompt_lines)

    # Extract prompt
    in_docstring = get_between(prompt_lines, docstring_start, docstring_end)
    if '"""' in in_docstring[0] or "'''" in in_docstring[0]:
        in_docstring[0] = in_docstring[0].replace('"""', "").replace("'''", "").strip()
    request = "Write a Python function which follows this instruction: " + " ".join([p.strip() for p in in_docstring])

    # Extract solution
    before_docstring = get_before(prompt_lines, docstring_start)
    after_docstring = sample["canonical_solution"].splitlines()
    solution = before_docstring + after_docstring
    # Gets rid of consecutive empty lines
    solution = [v for i, v in enumerate(solution) if v != "" or v != solution[i - 1]]
    solution = "\n".join(solution)

    return request, solution

In [6]:
def process_file(file_path: Path, out_path: Path, source: str):
    lines = file_path.read_text().splitlines()
    samples = list(map(json.loads, lines))

    output = []
    for sample in samples:
        prompt, solution = get_request_and_solution(sample)
        output.append({"INSTRUCTION": prompt, "RESPONSE": solution, "SOURCE": source})

    with open(out_path, "w") as f:
        for sample in output:
            f.write(json.dumps(sample))
            f.write("\n")

In [7]:
for file_path, out_path, source in zip(FILE_PATHS, OUT_PATHS, DATA_SOURCES):
    process_file(file_path, out_path, source)

Display a sample output from HumanEval

In [8]:
sample = json.loads(OUT_PATHS[0].read_text().splitlines()[0])

print("Prompt")
print(sample["INSTRUCTION"])
print()
print("Solution")
print(sample["RESPONSE"])

Prompt
Write a Python function which follows this instruction: Check if in given list of numbers, are any two numbers closer to each other than given threshold.

Solution
from typing import List

def has_close_elements(numbers: List[float], threshold: float) -> bool:
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = abs(elem - elem2)
                if distance < threshold:
                    return True

    return False


Display a sample output from MBPP

In [9]:
sample = json.loads(OUT_PATHS[1].read_text().splitlines()[0])

print("Prompt")
print(sample["INSTRUCTION"])
print()
print("Solution")
print(sample["RESPONSE"])

Prompt
Write a Python function which follows this instruction:  Write a function to find the shared elements from the given two lists.

Solution
def similar_elements(test_tup1, test_tup2):
  res = tuple(set(test_tup1) & set(test_tup2))
  return (res) 


Upload to HuggingFace

In [ ]:
from datasets import Dataset

humaneval_mbpp_codegen_qa_ds = Dataset.from_json([str(p) for p in OUT_PATHS])
humaneval_mbpp_codegen_qa_ds.push_to_hub("OllieStanley/humaneval-mbpp-codegen-qa")